In [ ]:
import OpenDartReader

api_key = '39849de683995a798ae30f7ceecf0bc4a7c923bf'

dart = OpenDartReader(api_key) 


# == 1. 공시정보 검색 ==
# 삼성전자의 정기보고서('A') 2019년
dart.list('005930', kind='A', start='2019-01-01', end='2019-12-31')

# 삼성전자의 모든 공시 리스트 (1999년 ~ 현재)
dart.list('005930') 

# 기업의 개황정보
dart.company('005930')

# 회사명에 삼성전자가 포함된 회사들에 대한 개황정보
dart.company_by_name('삼성전자')

# 삼성전자 사업보고서 (2018.12) 원문 텍스트
xml_text = dart.document('20190401004781')


# ==== 2. 사업보고서 ====
# 삼성전자(005930), 배당관련 사항, 2018년
dart.report('005930', '배당', 2018) 

# 서울반도체(046890), 최대주주 관한 사항, 2018년
dart.report('046890', '최대주주', 2018) 

# 서울반도체(046890), 임원 관한 사항, 2018년
dart.report('046890', '임원', 2018) 


# ==== 3. 상장기업 재무정보 ====
# 삼성전자 2018 재무제표 
dart.finstate('삼성전자', 2018) # 사업보고서

# 삼성전자 2018Q1 재무제표
dart.finstate('삼성전자', 2018, reprt_code='11013')

# 여러종목 한번에
dart.finstate('00126380,00164779,00164742', 2018)
dart.finstate('005930, 000660, 005380', 2018)
dart.finstate('삼성전자, SK하이닉스, 현대자동차', 2018)

# 단일기업 전체 재무제표 (삼성전자 2018 전체 재무제표)
dart.finstate_all('005930', 2018)

# 재무제표 XBRL 원본 파일 저장 (삼성전자 2018 사업보고서)
dart.finstate_xml('20190401004781', save_as='삼성전자_2018_사업보고서_XBRL.zip')

# XBRL 표준계정과목체계(계정과목)
dart.xbrl_taxonomy('BS1')


# ==== 4. 지분공시 ====
# 대량보유 상황보고 (종목코드, 종목명, 고유번호 모두 지정 가능)
dart.major_shareholders('삼성전자')

# 임원ㆍ주요주주 소유보고 (종목코드, 종목명, 고유번호 모두 지정 가능)
dart.major_shareholders_exec('005930')


# ==== 5. 확장 기능 ====
# 지정한 날짜의 공시목록 전체
dart.list_date('2020-01-03')

# 지정한 날짜의 공시목록 전체 (시간 정보 포함)
dart.list_date_ex('2020-01-03')

# 개별 문서 제목과 URL
rcp_no = '20190401004781' # 삼성전자 2018년 사업보고서
dart.sub_docs(rcp_no)

# 제목이 잘 매치되는 순서로 소트
dart.sub_docs('20190401004781', match='사업의 내용')

# 첨부 문서 제목과 URL
dart.attach_docs(rcp_no)

# 제목이 잘 매치되는 순서로 소트
dart.attach_docs(rcp_no, match='감사보고서')

# 첨부 파일 제목과 URL
dart.attach_files(rcp_no)


In [ ]:

import OpenDartReader

api_key = '39849de683995a798ae30f7ceecf0bc4a7c923bf'

dart = OpenDartReader(api_key) 


# == 1. 공시정보 검색 ==
# 삼성전자의 정기보고서('A') 2019년
dart.list('005930', kind='A', start='2019-01-01', end='2019-12-31')

# 삼성전자의 모든 공시 리스트 (1999년 ~ 현재)
dart.finstate('005930','2019') 

In [ ]:
### 필요한 라이브러리 불러오기
import OpenDartReader
import numpy as np    
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


api_key = '39849de683995a798ae30f7ceecf0bc4a7c923bf'
dart = OpenDartReader(api_key) 

def finance(company, From, To) : 

    ### Dictionary
    df_info = {}
     
    for year in range(From, To+1) : 

        ### 재무제표
        fs = dart.finstate(company, year, reprt_code='11011') # 재무제표(전체)
        ###"""
        ###reprt_code : '11013' = 1분기보고서, '11012' = 반기보고서, '11014' = 3분기보고서, '11011' = 사업보고서
        ###"""
        fs_is = fs[fs['sj_div'].str.contains('IS')] # 재무제표(전체) > 손익계산서
        fs_bs = fs[fs['sj_div'].str.contains('BS')] # 재무제표(전체) > 재무제표
        fs_equity = fs_bs[fs_bs['account_nm'].str.contains('자본총계')] # 재무제표(전체) > 재무제표 > 자본총계
        fs_assets = fs_bs[fs_bs['account_nm'].str.contains('자산총계')] # 재무제표(전체) > 재무제표 > 자산총계
        fs_revenue = fs_is[fs_is['account_nm'].str.contains('매출액')] # 재무제표(전체) > 손익계산서 > 매출액
        fs_income = fs_is[fs_is['account_nm'].str.contains('영업이익')] # 재무제표(전체) > 손익계산서 > 영업이익
        fs_profit = fs_is[fs_is['account_nm'].str.contains('당기순이익')] # 재무제표(전체) > 손익계산서 > 당기순이익

        ### 재무제표 전체(재무상태표, 손익계산서, 자본변동표, 현금흐름표 등)
        fs_all = dart.finstate_all(company, year) # 재무제표(전체)
        fs_cf = fs_all[fs_all['sj_div'].str.contains('CF')] # 재무제표(전체) > 현금흐름표
        fs_OCF = fs_cf[fs_cf['account_id'].str.contains('CashFlowsFromUsedInOperatingActivities')] # 재무제표(전체) > 현금흐름표 > 영업활동현금흐름
        fs_aucqusition = fs_cf[fs_cf['account_nm'].str.contains('취득')] # 재무제표(전체) > 현금흐름표 >  '취득'
        fs_CAPEX_P = fs_aucqusition[fs_aucqusition['account_nm'].str.contains('유형자산')] # 재무제표(전체) > 현금흐름표 >  유형자산의취득
        fs_CAPEX_I = fs_aucqusition[fs_aucqusition['account_nm'].str.contains('무형자산')] # 재무제표(전체) > 현금흐름표 >  무형자산의취득

        ### 배당에 관한 사항
        dv = dart.report(company, '배당', year) # 배당에 관한 사항
        dv_dps = dv[ dv['se'] == '주당 현금배당금(원)' ] # 배당에 관한 사항 > 주당 배당금
        dv_eps = dv[ dv['se'].str.contains('주당순이익') ] # 배당에 관한 사항 > 주당 순이익
        dv_yield = dv[ dv['se'].str.contains('현금배당수익률') ] # 배당에 관한 사항 > 배당수익률
        dv_TD = dv[ dv['se'].str.contains('현금배당금총액') ] # 배당에 관한 사항 > 배당금 총액

   
        equity_this = int(fs_equity[['thstrm_amount']].iloc[0,0].replace(',', '').strip()) # 당해년도 자본총계
        equity_last = int(fs_equity[['frmtrm_amount']].iloc[0,0].replace(',', '').strip()) # 직전년도 자본총계
        equity = (equity_this + equity_last ) / 2 # 평균자본총계 
        assets_this = int(fs_assets[['thstrm_amount']].iloc[0,0].replace(',', '').strip()) # 당해년도 자본총계
        assets_last = int(fs_assets[['frmtrm_amount']].iloc[0,0].replace(',', '').strip()) # 직전년도 자본총계
        assets = (assets_this + assets_last ) / 2 # 평균자산총계
        revenue = int(fs_revenue[['thstrm_amount']].iloc[0,0].replace(',', '')) # 매출액
        income = int(fs_income[['thstrm_amount']].iloc[0,0].replace(',', '')) # 영업이익
        profit = int(fs_profit[['thstrm_amount']].iloc[0,0].replace(',', '')) # 순이익
        DPS = int( dv_dps[ ['thstrm'] ].iloc[0, 0].replace(',', '').strip() ) # 주당 배당금
        EPS = int( dv_eps[ ['thstrm'] ].iloc[0, 0].replace(',', '').strip() ) # 주당 순이익
        Yield = int( dv_yield[ ['thstrm'] ].iloc[0, 0].replace('.', '').strip() ) / 10000 # 배당수익률
        TD = int( dv_TD[ ['thstrm'] ].iloc[0, 0].replace(',', '').strip() ) # 배당금 총액
        OCF = int(fs_OCF[['thstrm_amount']].iloc[0,0]) # 당해년도
        CAPEX_P = int(fs_CAPEX_P[['thstrm_amount']].iloc[0,0]) # 유형자산의취득
        CAPEX_I = int(fs_CAPEX_I[['thstrm_amount']].iloc[0,0]) #  무형자산의취득
        CAPEX = CAPEX_P + CAPEX_I
        ROE = profit / equity # 자기자본수익률
        ROA = profit / assets # 자산수익률
        FCF = OCF - CAPEX # 잉여현금흐름
        OPM = income / revenue # 영업이익률
        NPM = profit / revenue # 순이익률
        PR = DPS / EPS # 배당성향
        
        ### dictionary에 담기
        df_info[str(year)] = \
        f'{revenue:,}'[:-8], \
        f'{income:,}'[:-8], \
        f'{profit:,}'[:-8], \
        f'{OCF:,}'[:-8], \
        f'{FCF:,}'[:-8], \
        f'{TD:,}', \
        f'{OPM:.1%}', \
        f'{NPM:.1%}', \
        f'{ROE:.1%}', \
        f'{ROA:.1%}', \
        f'{EPS:,}', \
        f'{DPS:,}', \
        f'{PR:.1%}', \
        f'{Yield:.1%}', \
        # f'{value:,}'[:-8] : ###,, / f'{value:.1%}' : #.#%
        
    ### 출력 ###
    df = pd.DataFrame(data = df_info, \
                      index = ['Revenue', 'Operating Profit', 'Net Profit', 
                               'Operating Cash Flow', 'Free Cash Flow', 'Total Dividend',
                               'Operating Profit Margin', 'Net Profit Margin', 
                               'Return on Equity', 'Return on Assets', 
                              'EPS', 'DPS', 'Payout Ratio', 'Yield'])

    company_nm = dart.company(company)['stock_name'] # 종목명
    company_nm_eng = dart.company(company)['corp_name_eng'] # 회사 영문명
    company_stock_code = dart.company(company)['stock_code'] # 종목코드    
    print(company_nm, company_nm_eng, company_stock_code, '(단위 : %, 백만원, 원)')
    return df

finance('삼성전자', 2015, 2020)

In [11]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import requests
from bs4 import BeautifulSoup

corp_code = '206650'
for page in range(1, 21):
    url_tmp = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=%s'
    url = url_tmp % (corp_code, page)
    html = requests.get(url).text
    html = html.replace('\n', '')
    html = html.replace('\t', '')
    html = html.replace(',', '')
    html = html.replace('.', '-')
    soup = BeautifulSoup(html, 'lxml')
   
    span_data = soup.find_all('span')
    td_data = soup.find_all('td')
    tr_data = soup.find_all('tr')
    th_data = soup.find_all('th')
   
    if page == 1:
        columns_list = []
        for i in range(len(th_data)):
            i_text = th_data[i].text
            columns_list.append(i_text)
        stock_price_all_array = np.array([columns_list])
   
    rest_page = 0
    stock_price_list = []
    for i in range(1, len(span_data)):
        a = span_data[i].text
        stock_price_list.append(a)
        if page > 1 and stock_price_list[0] == stock_price_all_array[-10,0]:
            rest_page = (21 - page)*10
            stock_price_array = np.zeros((rest_page,7))
            stock_price_all_array = np.append(stock_price_all_array, stock_price_array, axis=0)
            break
        if i % 7 == 0:
            stock_price_array = np.array([stock_price_list])
            stock_price_all_array = np.append(stock_price_all_array, stock_price_array, axis=0)
            stock_price_list = []   
    if rest_page:
        break
    elif len(span_data) < 70:
        rest_row = int((70 - len(span_data))/7+1)
        stock_price_array = np.zeros((rest_row, 7))
        stock_price_all_array = np.append(stock_price_all_array, stock_price_array, axis=0)  
        
        rest_page = (21 - page-1)*10
        stock_price_array = np.zeros((rest_page,7))
        stock_price_all_array = np.append(stock_price_all_array, stock_price_array, axis=0)
        break
   
stock_price_all_array = np.delete(stock_price_all_array, 0, 0)
stock_price_all_df = DataFrame(stock_price_all_array, columns=columns_list).set_index('날짜')

print(stock_price_all_df.head(10))



ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 7

In [10]:
url

'http://finance.naver.com/item/sise_day.nhn?code=206650&page=1'

In [29]:
headers = {'Content-Type': 'text/html; charset=euc-kr'}

html = requests.post('http://finance.naver.com/item/sise_day.nhn?code=206650&page=1',headers=headers)
#html = requests.post('http://finance.naver.com/item/sise_day.nhn?code=206650')
print(html.text)


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">
<title>네이버 :: 세상의 모든 지식, 네이버</title>

<style type="text/css">
.error_content * {margin:0;padding:0;}
.error_content img{border:none;}
.error_content em {font-style:normal;}
.error_content {width:410px; margin:80px auto 0; padding:57px 0 0 0; font-size:12px; font-family:"나눔고딕", "NanumGothic", "돋움", Dotum, AppleGothic, Sans-serif; text-align:left; line-height:14px; background:url(https://ssl.pstatic.net/static/common/error/090610/bg_thumb.gif) no-repeat center top; white-space:nowrap;}
.error_content p{margin:0;}
.error_content .error_desc {margin-bottom:21px; overflow:hidden; text-align:center;}
.error_content .error_desc2 {margin-bottom:11px; padding-bottom:7px; color:#888; line-height:18px; border-bottom:1px solid #eee;}
.error_content .error_desc3 {clear:both; color:#888;}
.error_content .error_desc3

In [ ]:
import requests
import time
import pandas as pd
 
from bs4 import BeautifulSoup as bs
from datetime import datetime
 
 
def exportData(code, startdate, enddate):
    
    isEnd = False
    res = {
        'index': [],
        'data': []
    }
    
    print("Start")
    
    page = 1
    while isEnd != True:
        url = "https://finance.naver.com/item/sise_day.nhn?code="+code+"&amp;page="+ str(page)
        response = requests.get(url)
        html = bs(response.text, 'html.parser')
        
        # parse
        trList = html.find_all("tr", {"onmouseover":"mouseOver(this)"})
        for tr in trList:
            tdList = tr.find_all('td')
#             print(tdList[0].text.strip())  # 날짜
#             print(tdList[1].text.strip())  # 종가
#             print(tdList[2].text.strip())  # 전일비
#             print(tdList[3].text.strip())  # 시가
#             print(tdList[4].text.strip())  # 고가
#             print(tdList[5].text.strip())  # 저가
#             print(tdList[6].text.strip())  # 거래량
        
            date = tdList[0].text.strip()  # 날짜
            closePrice = int(tdList[1].text.strip().replace(',', ''))  # 종가
            diffPrice = int(tdList[2].text.strip().replace(',', ''))  # 전일비
            openPrice = int(tdList[3].text.strip().replace(',', ''))  # 시가
            highPrice = int(tdList[4].text.strip().replace(',', ''))  # 고가
            lowPrice = int(tdList[5].text.strip().replace(',', ''))  # 저가
            volume = int(tdList[6].text.strip().replace(',', ''))  # 거래량
            
            target = datetime.fromisoformat(date.replace('.', '-'))
            if target < start:
                isEnd = True
                break
            elif target < end and target > start:
                print(target)
                # insert index
                res['index'].insert(0, date)
                
                # insert data with ["High","Low","Open","Close","Volume","Adj Close"]
                res['data'].insert(0, [highPrice, lowPrice, openPrice, closePrice, volume])
                        
        page += 1
        time.sleep(2)
        
    df = pd.DataFrame(data=res['data'], index=res['index'])
    df.to_json(code+'.json', orient='split', date_format='index')
    
    print("Start - Done")
    
code = "005930"
start = datetime(2019, 1, 1)
end = datetime(2019, 1, 31)
 
exportData(code, start, end)

In [ ]:
url